In [2]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory
from chromadb.config import Settings
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from textblob import TextBlob
import chromadb
import os
import re
import pickle
import random

In [3]:
crisis_score = 0

In [4]:
#calling api key and loading opeanai
os.environ['OPENAI_API_KEY'] = ""
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

In [5]:
#loading the documents
def load_doc(doc):
    loader = CSVLoader(doc)
    doc = loader.load()
    print(f"length of document {len(doc)}")
    print(f"content of document {doc[0].page_content}")
    return doc

In [6]:
#preprocessing the documents
def preprocessing(docs):
    for doc in docs:
        if doc.page_content:
            # Remove extra whitespace
            text = ' '.join(doc.page_content.split())
            doc.page_content = text
    return docs

In [7]:
#splitting document into chunks
def split_documents(clean):
    spliter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 200,
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
        length_function = len 
    )
    chunk = spliter.split_documents(clean)
    print(f"length of split document: {len(chunk)}")
    return chunk

In [19]:

def indexing1(chunks):
    try:
        # Ensure the persist directory exists
        persist_directory = "./chroma_db"
        os.makedirs(persist_directory, exist_ok=True)

        # Initialize the embedding model
        embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

        # Create a new client using the updated method
        client = chromadb.PersistentClient(path=persist_directory)

        # Create and persist the Chroma vector store
        vectorstore = Chroma.from_documents(
            documents=chunks,
            embedding=embedding,
            client=client,
            collection_name="my_collection"  # You can change this name as needed
        )
        
        print("Vector store created and persisted")
        return vectorstore
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

In [8]:
#embedding and indexing
def indexing(chunks, save_path="./faiss_index"):
    try:
        # Initialize the embedding model
        embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

        # Create and persist the Chroma vector store
        vectorstore = FAISS.from_documents(
            documents=chunks,
            embedding=embedding,
        ) 
        
        # Save the index
        vectorstore.save_local(save_path)       
        print("Vector store created and persisted")
        return vectorstore
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None


In [35]:
def load_vectorstore(save_path="./faiss_index"):
    try:
        # Initialize the embedding model
        #embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        
        # Load the index
        vectorstore = FAISS.load_local(save_path, allow_dangerous_deserialization=True)
        
        print("Vector store loaded")
        return vectorstore
    except Exception as e:
        print(f"Error occurred while loading vector store: {str(e)}")
        return None

In [9]:
#loading vector store from faiss
class LazyLoadedFAISS:
    def __init__(self, save_path):
        self.save_path = save_path
        self.vectorstore = None
        self.embedding = None

    def load(self):
        if self.vectorstore is None:
            self.embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
            self.vectorstore = FAISS.load_local(self.save_path, embeddings=self.embedding, allow_dangerous_deserialization=True)
        return self.vectorstore

    def similarity_search(self, query, *args, **kwargs):
        return self.load().similarity_search(query, *args, **kwargs)

    def add_documents(self, documents):
        return self.load().add_documents(documents)

def load_vectorstore(save_path="./faiss_index"):
    try:
        lazy_vectorstore = LazyLoadedFAISS(save_path)
        print("Lazy-loaded vector store initialized")
        return lazy_vectorstore
    except Exception as e:
        print(f"Error occurred while initializing lazy-loaded vector store: {str(e)}")
        return None

In [10]:


def retriever(vectorstore):
    template="""You are an AI assistant trained to provide supportive responses and practical advice for a range of issues, from everyday stress to more severe emotional distress. Your primary goals are:

        1. For minor issues (like exam stress, daily worries), provide practical advice, coping strategies, and emotional support.
        2. For moderate issues, offer more in-depth support and coping strategies, while gently suggesting professional help as an option.
        3. For severe issues (clear signs of depression, suicidal thoughts), strongly encourage seeking professional help while providing immediate emotional support.

        Always maintain a compassionate and supportive tone. Offer specific, actionable advice when possible.

        Context: {context}
        Chat History: {chat_history}
        Human: {question}

        AI Assistant:"""

    custom_prompt = ChatPromptTemplate.from_template(template)
    
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key='answer'
    )
    
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": custom_prompt},
        return_source_documents=True,
        return_generated_question=True,
    )
    
    print('Database retriever created')
    return chain

In [1]:
def chat(chain):
    while True:
        query = input('You: ')
        if query.lower() in ['exit', 'quit', 'bye']:
            print("AI: Goodbye! Take care.")
            break
        result = chain.invoke({"question": query})
        ai_response = result['answer']
        print(f"AI: {ai_response}")

In [11]:
#sentimentl analysis
def sentiment_analysis(sentiment):
    text = TextBlob(sentiment)
    return text

In [12]:
def check_risk_keywords(text):
    risk_keywords = ['suicide', 'kill myself', 'end it all', 'no reason to live']
    return any(keyword in text.lower() for keyword in risk_keywords)

In [13]:
def update_criticly_score(text, sentiment):
    global crisis_score
    if check_risk_keywords(text):
        crisis_score +=2
    if sentiment.polarity < -0.5:
        crisis_score += 1
    return crisis_score

In [14]:
def provide_helpline_info():
    return """
    If you're feeling suicidal, please reach out for help:
    - National Suicide Prevention Lifeline: 1-800-273-8255
    - Crisis Text Line: Text HOME to 741741
    Remember, you're not alone, and help is available 24/7.
    """

In [15]:
def chat_loop(retriever_chain):
    global crisis_score
    print("AI: Hello! I'm here to talk. How are you feeling today?")
    
    while True:
        user_input = input("You: ")
        
        sentiment = sentiment_analysis(user_input)
        crisis_score = update_criticly_score(user_input, sentiment)
        
        if crisis_score > 5:
            print("AI:", provide_helpline_info())
            crisis_score = 0
        
        result = retriever_chain({"question": user_input})
        ai_response = result['answer']
        print("AI:", ai_response)
        
        # Generate follow-up with a 30% chance
        # if random.random() < 0.3 and not any(word in user_input.lower() for word in ['exit', 'quit', 'bye']):
        #     follow_up = generate_follow_up(ai_response, retriever_chain.memory.chat_memory)
        #     print("AI:", follow_up)
        
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("AI: Take care. Remember, help is always available if you need it.")
            break

In [64]:
file = 'health.csv'
doc = load_doc(file)

length of document 500
content of document Scenario_ID: 1
User_Statement: "I can't stop thinking about making mistakes. It's like my brain is on replay, and I can't focus on anything else."
Counselor_Response: "It sounds like you're experiencing some intrusive thoughts. This can be very distressing. It's important to remember that everyone makes mistakes, and it's normal to have doubts. Have you tried any techniques to manage these thoughts, like mindfulness or distraction? We can explore strategies together to help you cope."
Mental_Health_Issue: Obsessive-Compulsive Disorder (OCD)
Explanation: The user describes symptoms of obsessive-compulsive disorder, specifically intrusive thoughts. The counselor acknowledges the distress and offers potential coping mechanisms, encouraging the user to seek further support.


In [65]:
clean =preprocessing(doc)

In [66]:
chunk = split_documents(clean)

length of split document: 500


In [67]:
indexes = indexing(chunk)

Vector store created and persisted


In [37]:
#vectorstore = load_vectorstore()

Error occurred while loading vector store: load_local() missing 1 required positional argument: 'embeddings'


In [25]:
# query = "I have stress in relationship and i dont know how to deal with it"
# result = indexes.similarity_search(query)
# for result in result:
#     print(result, "\n")

page_content='Scenario_ID: 139 User_Statement: "I've been feeling really anxious about my relationship. I'm scared it might end." Counselor_Response: "Relationship anxiety can be very distressing, and I'm sorry you're experiencing this. It's important to communicate your feelings and concerns with your partner. Have you considered having an open conversation about your worries? Let's explore some ways to help you manage this anxiety and strengthen your relationship." Mental_Health_Issue: Relationship Anxiety Explanation: This scenario addresses relationship anxiety. The user's statement reflects fear about the potential end of a relationship. The counselor acknowledges the distress and suggests open communication with the partner, offering support to manage the anxiety and strengthen the relationship.' metadata={'source': 'health.csv', 'row': 138} 

page_content='Scenario_ID: 339 User_Statement: "I've been feeling really anxious about my relationship. I'm scared it might end." Counselo

In [18]:
lazy_vectorstore = load_vectorstore()
query = "I have stress in relationship and i dont know how to deal with it"
result = lazy_vectorstore.similarity_search(query)
for result in result:
    print(result, "\n")

Lazy-loaded vector store initialized


c:\Users\Ather Ali\anaconda3\envs\fyp\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


page_content='Scenario_ID: 139 User_Statement: "I've been feeling really anxious about my relationship. I'm scared it might end." Counselor_Response: "Relationship anxiety can be very distressing, and I'm sorry you're experiencing this. It's important to communicate your feelings and concerns with your partner. Have you considered having an open conversation about your worries? Let's explore some ways to help you manage this anxiety and strengthen your relationship." Mental_Health_Issue: Relationship Anxiety Explanation: This scenario addresses relationship anxiety. The user's statement reflects fear about the potential end of a relationship. The counselor acknowledges the distress and suggests open communication with the partner, offering support to manage the anxiety and strengthen the relationship.' metadata={'source': 'health.csv', 'row': 138} 

page_content='Scenario_ID: 339 User_Statement: "I've been feeling really anxious about my relationship. I'm scared it might end." Counselo

In [19]:
#The lazy_vectorstore.load() call is important because your LazyLoadedFAISS class needs to actually load the vectorstore before we can use it.
db = retriever(lazy_vectorstore.load())

Database retriever created


In [130]:
chat(db)

In [20]:
chat_loop(db)

AI: Hello! I'm here to talk. How are you feeling today?


c:\Users\Ather Ali\anaconda3\envs\fyp\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


AI: Hello! How are you feeling today?
AI: I'm here and ready to help you. How are you feeling today?
AI: I'm here to support you with anything you need. Feel free to share what's on your mind, and I'll do my best to help.
AI: Take care. Remember, help is always available if you need it.
